In [1]:
import pandas as pd
import json

### HealthCareMagic-100k-en Dataset

In [2]:
hc_ds = pd.read_json("dataset/HealthCareMagic-100k-en.jsonl", lines=True)

In [3]:
hc_ds.head(1)

,text
0,<human>: I woke up this morning feeling the wh...


In [4]:
def parse_hc_text(row):
    text = row["text"]

    try:
        human= text.split("<human>:")[1].split("<bot>:")[0].strip()
        bot= text.split("<bot>:")[1].strip()

        return {
            "instruction" :  "Answer the following medical question in a clear and concise way.",
            "input" : human,
            "output" : bot
        }
    except IndexError:
        return None

In [48]:
hc_ds_formatted = hc_ds.apply(parse_hc_text, axis=1)

In [7]:
hc_ds_formatted.info()

<class 'pandas.core.series.Series'>
RangeIndex: 112165 entries, 0 to 112164
Series name: None
Non-Null Count   Dtype 
--------------   ----- 
112165 non-null  object
dtypes: object(1)
memory usage: 876.4+ KB


In [25]:
hc_dc_formatted = [x for x in hc_ds_formatted if x is not None]

In [27]:
with open("dataset/hc_instruction_format.jsonl", "w", encoding="utf-8") as f:
    for item in hc_ds_formatted:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"Converted {len(hc_ds_formatted)} examples to instruction format.")

Converted 112165 examples to instruction format.


### medquad Dataset

In [9]:
mq_ds = pd.read_csv("dataset\medquad.csv")

In [11]:
def parse_mq_text(row):
    try:
        human = row["question"]
        bot = row["answer"]

        return {
            "instruction" :  "Answer the following medical question in a clear and concise way.",
            "input" : human,
            "output" : bot
        }
    except IndexError:
        None

In [46]:
mq_ds_formatted = mq_ds.apply(parse_mq_text, axis=1)

In [40]:
mq_ds_formatted = [x for x in mq_ds_formatted if x is not None]

In [42]:
with open("dataset/mq_instruction_format.jsonl", "w", encoding="utf-8") as f:
    for item in mq_ds_formatted:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"Converted {len(mq_ds_formatted)} examples to instruction format.")

Converted 16412 examples to instruction format.


### Combine Dataset

In [56]:
combined_df = pd.concat([hc_ds_formatted, mq_ds_formatted], ignore_index=True)

In [61]:
combined_df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 128577 entries, 0 to 128576
Series name: None
Non-Null Count   Dtype 
--------------   ----- 
128577 non-null  object
dtypes: object(1)
memory usage: 1004.6+ KB


In [ ]:
import random

random.shuffle(combined_df)

In [60]:
with open("dataset/medical_qa_combined_shuffled.jsonl", "w", encoding="utf-8") as f:
    for item in combined_df:
        f.write(json.dumps(item) + "\n")